# Built a simple bot

## How to integrate LLM into our Graph

In [1]:
import os
from typing import TypedDict, List, Dict, Any, Optional
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
class AgentState(TypedDict): 
    messages: List[HumanMessage]

llm =ChatOpenAI(model="gpt-4o")

In [5]:
def process(state: AgentState) -> AgentState:
    response =llm.invoke(state["messages"])
    print(f"\nAI:{response.content}")
    return state

In [6]:
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process") 
graph.add_edge("process", END)

agent = graph.compile()

In [11]:
user_input= input("Enter: ")
while user_input!= "exit":
    agent.invoke({'messages':[HumanMessage(content=user_input)]})
    user_input= input("Enter: ")

Enter:  hi



AI:Hello! How can I assist you today?


{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]}

Try giving input and asking the questions. It cannot answer your questions based on your questions

## Let's make an agent

In [21]:
import os
from typing import TypedDict, List, Dict, Any, Optional, Union
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv

In [19]:
load_dotenv()

True

In [23]:
class AgentState(TypedDict): 
    messages: List[Union[HumanMessage, AIMessage]]

llm =ChatOpenAI(model="gpt-4o")

def process(state: AgentState) -> AgentState:
    """ Thid nod will solve the request you input"""
    response =llm.invoke(state["messages"])
    state["messages"].append(AIMessage(content=response.content))
    print(f"\nAI:{response.content}")
    print("CURRENT STATE:", state["messages"])
    return state

In [ ]:
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process") 
graph.add_edge("process", END)

agent = graph.compile()

# starting from here it gets different from the previous one

conversation_history=[]

user_input= input("Enter: ")
while user_input!= "exit":
    conversation_history.append(HumanMessage(content=user_input))
    result= agent.invoke({'messages':conversation_history})
    conversation_history= result["messages"]
    user_input= input("Enter: ")

### with this version if we exit the bot, it is not going to remember our name again when we run it.

Let's improve it

In [ ]:
import os
from typing import TypedDict, List, Dict, Any, Optional, Union
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
class AgentState(TypedDict): 
    messages: List[Union[HumanMessage, AIMessage]]

llm =ChatOpenAI(model="gpt-4o")

def process(state: AgentState) -> AgentState:
    """ Thid nod will solve the request you input"""
    response =llm.invoke(state["messages"])
    state["messages"].append(AIMessage(content=response.content))
    print(f"\nAI:{response.content}")
    print("CURRENT STATE:", state["messages"])
    return state

In [ ]:
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process") 
graph.add_edge("process", END)

agent = graph.compile()

# starting from here it gets different from the previous one

conversation_history=[]

user_input= input("Enter: ")
while user_input!= "exit":
    conversation_history.append(HumanMessage(content=user_input))
    result= agent.invoke({'messages':conversation_history})
    conversation_history= result["messages"]
    user_input= input("Enter: ")

#this part is saving our conversation as a txt file
with open("logging.txt" , "w") as file:
    file.write("Your Conversation Log:\n")
    for message in conversation_history:
        if isinstance(message, HumanMessage):
            file.write(f"You: {message.content}\n")
        elif isinstance(message, AIMessage):
            file.write(f"AI: {message.content}\n\n")
        file.write("End of Conversation")
print("Conversation saved to logging.txt)

### Our current state is going to be longer and longer. And using that many tokens will increase the price for that. We need to me clever about it

We can write a code like if the number of the human messages exceeds 5 then remove the first message